## Exercice 1

In [15]:
!pip install confluent-kafka

In [45]:
import random
from random import randrange
import datetime
import uuid


def random_date(start,l):
   current = start
   while l >= 0:
      current = current + datetime.timedelta(minutes=randrange(5))
      yield current
      l-=1

def random_user(user_number):
  users = []
  for i in range(user_number):
    users.append(str(uuid.uuid4())[:8])
  return users

user_db = random_user(20)



In [46]:

actions = [
    "view",
    "search",
    "add_to_cart",
    "remove_from_cart",
    "checkout",
    "login",
    "logout",
    "register",
    "download",
    "scroll"
    "share",
    "go_to",
    "close"
]


pages = [
    "home",
    "about",
    "contact",
    "products",
    "product1",
    "product2",
    "cart",
    "checkout",
    "order_success",
    "account",
    "login",
    "register",
    "faq",
    "privacy_policy"
]


In [143]:
def generate_log_line(users, actions, pages, num_logs):

    active_users = {}
    users_arrived = []
    num_users = len(users)
    all_logs = []

    for t in random_date(datetime.datetime.now(),num_logs):

            arrival_probability = 0.3 if len(users_arrived) < num_users * 0.7 else 0.1

            if (len(users_arrived) <= num_users and
                random.random() < arrival_probability) :

                available = [u for u in users if u not in users_arrived]
                if available:
                    new_user = random.choice(available)
                    users_arrived.append(new_user)


                    all_logs.append({
                        'timestamp': f"{t}",
                        'user_id': new_user,
                        'action': 'go_to',
                        'page': 'home'
                    })


                    active_users[new_user] = {
                        'actions_done': 1,
                        'start_time': t,
                        'will_close': random.random() > 0.15
                    }

                    continue


            if active_users:
                active_user = random.choice(list(active_users.keys()))
                session = active_users[active_user]

                min_actions = random.randint(3,8)
                should_leave = (
                    session['actions_done'] >= min_actions
                )

                if should_leave:
                    if session['will_close']:
                        all_logs.append({
                            'timestamp': f"{t}",
                            'user_id': active_user,
                            'action': 'close',
                            'page': random.choice(pages)
                        })

                    del active_users[active_user]

                else:
                    normal_actions = [a for a in actions if a not in ['close']]

                    all_logs.append({
                        'timestamp':f"{t}",
                        'user_id': active_user,
                        'action': random.choice(normal_actions),
                        'page': random.choice(pages)
                    })

                    session['actions_done'] += 1
            else:
              if users_arrived:
                  fallback_user = random.choice(users_arrived)
                  all_logs.append({
                      'timestamp': t,
                      'user_id': fallback_user,
                      'action': random.choice(actions),
                      'page': random.choice(pages)
                  })



    try:
        with open('logs.txt', 'w') as f:
            for idx, log in enumerate(all_logs, 1):
                f.write(f"{idx} -- Timestamp : {log['timestamp']} - "
                       f"User_id : {log['user_id']} -- "
                       f"Action : {log['action']} -- "
                       f"Page : {log['page']}\n")
    except Exception as e:
        print(f"Erreur : {e}")

    print("Fichier log créé avec succès")

In [145]:
generate_log_line(user_db,actions,pages,200)

Fichier log créé avec succès


## Exercice 2

In [127]:
# Required connection configs for Kafka producer, consumer, and admin
config = {
    "bootstrap.servers":"pkc-60py3.europe-west9.gcp.confluent.cloud:9092",
"security.protocol":"SASL_SSL",
"sasl.mechanisms" : "PLAIN",
"sasl.username" : "FCLAEESPDSIC7UXR",
"sasl.password" : "cfltt3mqlGreq7TUTihrQURLGc89LrQPW//flojUU+1Npqh88mMXbE/OHMZvfWYQ",
"session.timeout.ms" : "45000",
"client.id" : "ccloud-python-client-c5b1ae8e-c274-4713-ad69-b0963c867f29"
}
topic="log_web"

In [147]:
from confluent_kafka import Producer

def produce(topic, config, key=None, value=None):
  producer = Producer(config)
  key = key
  value = value
  producer.produce(topic, key=key, value=value, callback=delivery_status)

  producer.poll(1)

def delivery_status(err, msg):
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()}")


In [148]:
import json
import time

def parse_line(line):
    try:
        parts = line.strip().split(' -- ')

        timestamp = parts[1].split(' : ', 1)[1].split(' - ')[0].strip()
        user_id = parts[1].split('User_id : ')[1].split(' --')[0].strip()
        action = parts[2].split(' : ')[1].strip()
        page = parts[3].split(' : ')[1].strip()

        return {
            'timestamp': timestamp,
            'user_id': user_id,
            'action': action,
            'page': page
        }
    except:
        return None


In [149]:
from confluent_kafka import Producer
import json
import time

producer = Producer(config) # Initialize the producer outside the loop

def delivery_status(err, msg):
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

with open('logs.txt', 'r') as f:
    lignes = f.readlines()
print(f"nombre de ligne de log: {len(lignes)}")
compteur = 0
for ligne in lignes:
    if len(ligne) != 0:
        try:

            log_data = parse_line(ligne)
            if log_data:
                key = log_data.get('user_id')
                value = json.dumps(log_data)
                print(key)
                try:
                  producer.produce(topic, key=key, value=value.encode('utf-8'), callback=delivery_status)
                  producer.poll(1)
                  compteur +=1
                except e:
                  print(e)
                time.sleep(1)

        except Exception as e:
            print(f"Error processing line: {ligne.strip()} - {e}")

producer.flush()

print(f"Tache finie, {compteur} lignes/{len(lignes)} lignes envoyées")

nombre de ligne de log: 193
a0aa0dd0
a0aa0dd0
Message delivered to log_web [0]
864064af
Message delivered to log_web [0]
9c2b2f06
Message delivered to log_web [2]
b446ca4a
Message delivered to log_web [1]
b446ca4a
Message delivered to log_web [2]
9c2b2f06
Message delivered to log_web [2]
b446ca4a
Message delivered to log_web [1]
9c2b2f06
Message delivered to log_web [2]
a0aa0dd0
Message delivered to log_web [1]
a0aa0dd0
Message delivered to log_web [0]
86e61f13
Message delivered to log_web [0]
9c2b2f06
Message delivered to log_web [1]
f1bd1c21
Message delivered to log_web [1]
1d6f1f65
Message delivered to log_web [2]
a0aa0dd0
Message delivered to log_web [0]
7844dd61
Message delivered to log_web [0]
f1bd1c21
Message delivered to log_web [0]
1d6f1f65
Message delivered to log_web [2]
f1bd1c21
Message delivered to log_web [0]
a0aa0dd0
Message delivered to log_web [2]
b446ca4a
Message delivered to log_web [0]
f1bd1c21
Message delivered to log_web [2]
864064af
Message delivered to log_web [

# 3


In [189]:
readconfig = {
    "bootstrap.servers": "pkc-60py3.europe-west9.gcp.confluent.cloud:9092",
    "security.protocol": "SASL_SSL",
    "sasl.mechanisms": "PLAIN",
    "sasl.username": "FCLAEESPDSIC7UXR",
    "sasl.password": "cfltt3mqlGreq7TUTihrQURLGc89LrQPW//flojUU+1Npqh88mMXbE/OHMZvfWYQ",
    "session.timeout.ms": "45000",
    "client.id": "ccloud-python-client",
    "group.id": "web_log-group-readerb",
    "auto.offset.reset": "earliest"
}

topic = "log_web"

In [190]:
from confluent_kafka import Consumer
import sys
import json

consumer = Consumer(readconfig)

try:
    consumer.subscribe(['log_web'])
except Exception as e:
    print(f"Error subscribing to topic {topic}: {e}")
    sys.exit()

all_msg = []
no_message_count = 0

try:
    while True:
        msg = consumer.poll(timeout=1.0)

        if msg is None:
            no_message_count += 1
            if no_message_count >= 10:
                break
            continue

        if msg.error() is None:
            key = msg.key().decode("utf-8")
            value = msg.value().decode("utf-8")

            print(f"Consumed message from topic {topic}: key = {key:12} value = {value:12}")

            all_msg.append(json.loads(value))
            no_message_count = 0

except Exception as e:
    print(e)

finally:
    consumer.close()
    print(f"Total messages: {len(all_msg)}")

Consumed message from topic log_web: key = a0aa0dd0     value = {"timestamp": "2025-10-16 21:16:47.779100", "user_id": "a0aa0dd0", "action": "go_to", "page": "home"}
Consumed message from topic log_web: key = a0aa0dd0     value = {"timestamp": "2025-10-16 21:16:47.779100", "user_id": "a0aa0dd0", "action": "view", "page": "faq"}
Consumed message from topic log_web: key = a0aa0dd0     value = {"timestamp": "2025-10-16 21:31:47.779100", "user_id": "a0aa0dd0", "action": "go_to", "page": "checkout"}
Consumed message from topic log_web: key = a0aa0dd0     value = {"timestamp": "2025-10-16 21:35:47.779100", "user_id": "a0aa0dd0", "action": "checkout", "page": "register"}
Consumed message from topic log_web: key = 1d6f1f65     value = {"timestamp": "2025-10-16 21:38:47.779100", "user_id": "1d6f1f65", "action": "go_to", "page": "home"}
Consumed message from topic log_web: key = a0aa0dd0     value = {"timestamp": "2025-10-16 21:42:47.779100", "user_id": "a0aa0dd0", "action": "scrollshare", "page

stats

In [193]:
from datetime import datetime
from collections import defaultdict


num_logs = len(all_msg)
unique_users = len(set(m['user_id'] for m in all_msg if 'user_id' in m))

sessions = defaultdict(dict)
for m in all_msg:
    user_id = m.get('user_id')
    action = m.get('action')
    page = m.get('page')
    timestamp_str = m.get('timestamp')

    if user_id and timestamp_str:
        try:
            ts = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            print(f"Warning: Could not parse timestamp: {timestamp_str}")
            continue

        if action == 'go_to' and page == 'home':
            sessions[user_id]['start'] = ts
        elif action == 'close':
            sessions[user_id]['end'] = ts

durations = [(s['end'] - s['start']).total_seconds()
             for s in sessions.values() if 'start' in s and 'end' in s]

avg_duration = sum(durations) / len(durations) if durations else 0



In [194]:
print(f"Nombre de logs: {num_logs}")
print(f"Utilisateurs uniques: {unique_users}")
print(f"Durée moyenne: {avg_duration:.2f}s")

Nombre de logs: 193
Utilisateurs uniques: 20
Durée moyenne: 7263.53s
